In [3]:
import pandas as pd

df = pd.read_csv("../data/raw/road_eqr_carpda.csv")
df.head()


ModuleNotFoundError: No module named 'pandas'

In [1]:
import pandas as pd

df = pd.read_csv("../data/raw/road_eqr_carpda.csv")
df.head()


,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,freq,Time frequency,unit,Unit of measure,mot_nrg,Motor energy,geo,Geopolitical entity (reporting),TIME_PERIOD,Time,OBS_VALUE,Observation value,OBS_FLAG,Observation status (Flag) V2 structure,CONF_STATUS,Confidentiality status (flag)
0,dataflow,ESTAT:ROAD_EQR_CARPDA$DEFAULTVIEW(1.0),New passenger cars by type of motor energy,A,Annual,NR,Number,TOTAL,Total,AL,Albania,2019,NaN,42681,NaN,NaN,NaN,NaN,NaN
1,dataflow,ESTAT:ROAD_EQR_CARPDA$DEFAULTVIEW(1.0),New passenger cars by type of motor energy,A,Annual,NR,Number,TOTAL,Total,AL,Albania,2020,NaN,43661,NaN,NaN,NaN,NaN,NaN
2,dataflow,ESTAT:ROAD_EQR_CARPDA$DEFAULTVIEW(1.0),New passenger cars by type of motor energy,A,Annual,NR,Number,TOTAL,Total,AL,Albania,2021,NaN,58208,NaN,NaN,NaN,NaN,NaN
3,dataflow,ESTAT:ROAD_EQR_CARPDA$DEFAULTVIEW(1.0),New passenger cars by type of motor energy,A,Annual,NR,Number,TOTAL,Total,AL,Albania,2022,NaN,49459,NaN,NaN,NaN,NaN,NaN
4,dataflow,ESTAT:ROAD_EQR_CARPDA$DEFAULTVIEW(1.0),New passenger cars by type of motor energy,A,Annual,NR,Number,TOTAL,Total,AL,Albania,2023,NaN,66004,NaN,NaN,NaN,NaN,NaN


In [2]:
import sys
print(sys.executable)


/Users/ritej/Documents/ev-europe-market/.venv/bin/python


In [2]:
from pathlib import Path
import pandas as pd
import re

DATA_DIR = Path("../data/raw")
OUT_DIR = Path("../data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

def load_eurostat(filepath: Path) -> pd.DataFrame:
    sep = "\t" if filepath.suffix.lower() == ".tsv" else ","
    df = pd.read_csv(filepath, sep=sep)

    # Harmoniser colonnes
    cols_lower = {c.lower(): c for c in df.columns}
    if "time" in cols_lower:
        df = df.rename(columns={cols_lower["time"]: "year"})
    elif "year" in cols_lower:
        df = df.rename(columns={cols_lower["year"]: "year"})

    # Large -> long si colonnes années
    year_like = [c for c in df.columns if re.fullmatch(r"\d{4}", str(c))]
    if year_like:
        id_cols = [c for c in df.columns if c not in year_like]
        df = df.melt(id_vars=id_cols, value_vars=year_like,
                     var_name="year", value_name="value")
    else:
        for c in df.columns:
            if c.lower() in ["values", "value", "obs_value"]:
                if c != "value":
                    df = df.rename(columns={c: "value"})
                break

    # Geo
    if "geo" not in df.columns:
        for c in df.columns:
            if c.lower().startswith("geo"):
                df = df.rename(columns={c: "geo"})
                break

    # Identifier colonne motorisation
    motor_col = None
    for c in df.columns:
        if c.lower() not in ["geo", "year", "value"] and re.search(r"motor|fuel|energy|engine|pow|drv|type", c, re.I):
            motor_col = c
            break
    if motor_col is None:
        raise ValueError("Colonne motorisation introuvable dans ce CSV.")

    # Nettoyage
    df["year"] = df["year"].astype(str).str.extract(r"(\d{4})").astype(int)
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df = df.dropna(subset=["value"])
    df = df.rename(columns={motor_col: "motor_energy_raw"})

    def normalize_motor(x: str) -> str:
        x = str(x).lower()
        if ("bev" in x) or ("battery" in x) or ("electric" in x and "hybrid" not in x and "plug" not in x):
            return "BEV"
        if ("plug-in" in x) or ("phev" in x):
            return "PHEV"
        if "hybrid" in x and "plug" not in x:
            return "HEV"
        if "diesel" in x:
            return "Diesel"
        if "petrol" in x or "gasoline" in x:
            return "Petrol"
        return "Other"

    df["motor_energy"] = df["motor_energy_raw"].map(normalize_motor)
    df = df.query("2015 <= year <= 2023")[["geo", "year", "motor_energy", "value"]]

    # Pivot et indicateurs
    pivot = df.pivot_table(index=["geo", "year"], columns="motor_energy",
                           values="value", aggfunc="sum").fillna(0)
    for col in ["BEV", "PHEV", "HEV", "Diesel", "Petrol", "Other"]:
        if col not in pivot.columns:
            pivot[col] = 0
    pivot["EV_total"] = pivot["BEV"] + pivot["PHEV"]
    pivot["Hybrid_total"] = pivot["HEV"] + pivot["PHEV"]
    pivot["All_fuels_total"] = pivot.sum(axis=1)

    return pivot.reset_index()

# Cherche le fichier CSV ou TSV
eurostat_file = None
for name in ["road_eqr_carpda.csv", "road_eqr_carpda.tsv"]:
    p = DATA_DIR / name
    if p.exists():
        eurostat_file = p
        break
if eurostat_file is None:
    raise FileNotFoundError("Fichier manquant : place road_eqr_carpda.csv (ou .tsv) dans data/raw/")

eu = load_eurostat(eurostat_file)
eu.to_csv(OUT_DIR / "eu_ev_by_country_year.csv", index=False)

print("✅ Créé: data/processed/eu_ev_by_country_year.csv")


ValueError: cannot convert float NaN to integer

In [3]:
from pathlib import Path
import pandas as pd
import re

DATA_DIR = Path("../data/raw")
OUT_DIR = Path("../data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

def load_eurostat(filepath: Path) -> pd.DataFrame:
    sep = "\t" if filepath.suffix.lower() == ".tsv" else ","
    df = pd.read_csv(filepath, sep=sep)

    # Harmoniser colonnes
    cols_lower = {c.lower(): c for c in df.columns}
    if "time" in cols_lower:
        df = df.rename(columns={cols_lower["time"]: "year"})
    elif "year" in cols_lower:
        df = df.rename(columns={cols_lower["year"]: "year"})

    # Large -> long si colonnes années
    year_like = [c for c in df.columns if re.fullmatch(r"\d{4}", str(c))]
    if year_like:
        id_cols = [c for c in df.columns if c not in year_like]
        df = df.melt(id_vars=id_cols, value_vars=year_like,
                     var_name="year", value_name="value")
    else:
        for c in df.columns:
            if c.lower() in ["values", "value", "obs_value"]:
                if c != "value":
                    df = df.rename(columns={c: "value"})
                break

    # Geo
    if "geo" not in df.columns:
        for c in df.columns:
            if c.lower().startswith("geo"):
                df = df.rename(columns={c: "geo"})
                break

    # Identifier colonne motorisation
    motor_col = None
    for c in df.columns:
        if c.lower() not in ["geo", "year", "value"] and re.search(r"motor|fuel|energy|engine|pow|drv|type", c, re.I):
            motor_col = c
            break
    if motor_col is None:
        raise ValueError("Colonne motorisation introuvable dans ce CSV.")

    # Nettoyage
    yrs = df["year"].astype(str).str.extract(r"(\d{4})")[0]
    df["year"] = yrs
    df = df.dropna(subset=["year"])        # supprime lignes vides
    df["year"] = df["year"].astype(int)

    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df = df.dropna(subset=["value"])
    df = df.rename(columns={motor_col: "motor_energy_raw"})

    # Normalisation des motorisations
    def normalize_motor(x: str) -> str:
        x = str(x).lower()
        if ("bev" in x) or ("battery" in x) or ("electric" in x and "hybrid" not in x and "plug" not in x):
            return "BEV"
        if ("plug-in" in x) or ("phev" in x):
            return "PHEV"
        if "hybrid" in x and "plug" not in x:
            return "HEV"
        if "diesel" in x:
            return "Diesel"
        if "petrol" in x or "gasoline" in x:
            return "Petrol"
        return "Other"

    df["motor_energy"] = df["motor_energy_raw"].map(normalize_motor)
    df = df.query("2015 <= year <= 2023")[["geo", "year", "motor_energy", "value"]]

    # Pivot et indicateurs
    pivot = df.pivot_table(index=["geo", "year"], columns="motor_energy",
                           values="value", aggfunc="sum").fillna(0)
    for col in ["BEV", "PHEV", "HEV", "Diesel", "Petrol", "Other"]:
        if col not in pivot.columns:
            pivot[col] = 0
    pivot["EV_total"] = pivot["BEV"] + pivot["PHEV"]
    pivot["Hybrid_total"] = pivot["HEV"] + pivot["PHEV"]
    pivot["All_fuels_total"] = pivot.sum(axis=1)

    return pivot.reset_index()

# Cherche le fichier CSV ou TSV
eurostat_file = None
for name in ["road_eqr_carpda.csv", "road_eqr_carpda.tsv"]:
    p = DATA_DIR / name
    if p.exists():
        eurostat_file = p
        break
if eurostat_file is None:
    raise FileNotFoundError("Fichier manquant : place road_eqr_carpda.csv (ou .tsv) dans data/raw/")

eu = load_eurostat(eurostat_file)
eu.to_csv(OUT_DIR / "eu_ev_by_country_year.csv", index=False)

print("✅ Créé: data/processed/eu_ev_by_country_year.csv")


✅ Créé: data/processed/eu_ev_by_country_year.csv


In [4]:
import pandas as pd
df = pd.read_csv("../data/processed/eu_ev_by_country_year.csv")
print(df.shape)
df.head()


(0, 11)


,geo,year,BEV,PHEV,HEV,Diesel,Petrol,Other,EV_total,Hybrid_total,All_fuels_total


In [5]:
import pandas as pd

raw = pd.read_csv("../data/raw/road_eqr_carpda.csv")
print("Colonnes :", list(raw.columns))
print("\nAperçu des 10 premières lignes :")
display(raw.head(10))

# Cherche la colonne 'valeur' (value/values/obs_value)
val_candidates = [c for c in raw.columns if c.lower() in ["value","values","obs_value","obs_value,"]]
print("\nColonnes possibles pour la valeur :", val_candidates)

# Cherche la colonne 'année' (time/year) et 'pays' (geo)
time_candidates = [c for c in raw.columns if c.lower() in ["time","year","time_period","date"]]
geo_candidates  = [c for c in raw.columns if c.lower().startswith("geo")]
print("Colonnes possibles pour l'année :", time_candidates)
print("Colonnes possibles pour le pays :", geo_candidates)

# Cherche la colonne 'motorisation' (fuel/engine/type...)
motor_candidates = [c for c in raw.columns if any(k in c.lower() for k in
                      ["motor","fuel","energy","engine","pow","drv","type"])]
print("Colonnes possibles pour la motorisation :", motor_candidates)

# Montre les valeurs uniques de chaque candidate (max 50)
for c in motor_candidates[:3]:
    print(f"\nExemples de valeurs pour '{c}':")
    print(raw[c].dropna().astype(str).unique()[:50])


Colonnes : ['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'freq', 'Time frequency', 'unit', 'Unit of measure', 'mot_nrg', 'Motor energy', 'geo', 'Geopolitical entity (reporting)', 'TIME_PERIOD', 'Time', 'OBS_VALUE', 'Observation value', 'OBS_FLAG', 'Observation status (Flag) V2 structure', 'CONF_STATUS', 'Confidentiality status (flag)']

Aperçu des 10 premières lignes :


,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,freq,Time frequency,unit,Unit of measure,mot_nrg,Motor energy,geo,Geopolitical entity (reporting),TIME_PERIOD,Time,OBS_VALUE,Observation value,OBS_FLAG,Observation status (Flag) V2 structure,CONF_STATUS,Confidentiality status (flag)
0,dataflow,ESTAT:ROAD_EQR_CARPDA$DEFAULTVIEW(1.0),New passenger cars by type of motor energy,A,Annual,NR,Number,TOTAL,Total,AL,Albania,2019,NaN,42681,NaN,NaN,NaN,NaN,NaN
1,dataflow,ESTAT:ROAD_EQR_CARPDA$DEFAULTVIEW(1.0),New passenger cars by type of motor energy,A,Annual,NR,Number,TOTAL,Total,AL,Albania,2020,NaN,43661,NaN,NaN,NaN,NaN,NaN
2,dataflow,ESTAT:ROAD_EQR_CARPDA$DEFAULTVIEW(1.0),New passenger cars by type of motor energy,A,Annual,NR,Number,TOTAL,Total,AL,Albania,2021,NaN,58208,NaN,NaN,NaN,NaN,NaN
3,dataflow,ESTAT:ROAD_EQR_CARPDA$DEFAULTVIEW(1.0),New passenger cars by type of motor energy,A,Annual,NR,Number,TOTAL,Total,AL,Albania,2022,NaN,49459,NaN,NaN,NaN,NaN,NaN
4,dataflow,ESTAT:ROAD_EQR_CARPDA$DEFAULTVIEW(1.0),New passenger cars by type of motor energy,A,Annual,NR,Number,TOTAL,Total,AL,Albania,2023,NaN,66004,NaN,NaN,NaN,NaN,NaN
5,dataflow,ESTAT:ROAD_EQR_CARPDA$DEFAULTVIEW(1.0),New passenger cars by type of motor energy,A,Annual,NR,Number,TOTAL,Total,AL,Albania,2024,NaN,82931,NaN,NaN,NaN,NaN,NaN
6,dataflow,ESTAT:ROAD_EQR_CARPDA$DEFAULTVIEW(1.0),New passenger cars by type of motor energy,A,Annual,NR,Number,TOTAL,Total,AT,Austria,2015,NaN,308555,NaN,NaN,NaN,NaN,NaN
7,dataflow,ESTAT:ROAD_EQR_CARPDA$DEFAULTVIEW(1.0),New passenger cars by type of motor energy,A,Annual,NR,Number,TOTAL,Total,AT,Austria,2016,NaN,329604,NaN,NaN,NaN,NaN,NaN
8,dataflow,ESTAT:ROAD_EQR_CARPDA$DEFAULTVIEW(1.0),New passenger cars by type of motor energy,A,Annual,NR,Number,TOTAL,Total,AT,Austria,2017,NaN,353320,NaN,NaN,NaN,NaN,NaN
9,dataflow,ESTAT:ROAD_EQR_CARPDA$DEFAULTVIEW(1.0),New passenger cars by type of motor energy,A,Annual,NR,Number,TOTAL,Total,AT,Austria,2018,NaN,341068,NaN,NaN,NaN,NaN,NaN



Colonnes possibles pour la valeur : ['OBS_VALUE']
Colonnes possibles pour l'année : ['TIME_PERIOD', 'Time']
Colonnes possibles pour le pays : ['geo', 'Geopolitical entity (reporting)']
Colonnes possibles pour la motorisation : ['Motor energy']

Exemples de valeurs pour 'Motor energy':
['Total']
